In [1]:
# !pip install gpxpy matplotlib

In [9]:
import gpxpy
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import folium

In [42]:
# Define the polynomial function for fuel consumption based on the slope (E)
def get_fuel_consumption(slope):
    return 6 + 0.766 * slope + 0.0646 * slope**2


def extract_route_data(gpx_file_path):
    with open(gpx_file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    latitudes = []
    longitudes = []
    elevations = []
    distances = []
    cumulative_distance = 0
    previous_point = None

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                latitudes.append(point.latitude)
                longitudes.append(point.longitude)
                elevations.append(point.elevation)
                if previous_point is not None:
                    distance = point.distance_2d(previous_point)
                    cumulative_distance += distance
                else:
                    distance = 0
                distances.append(cumulative_distance)
                previous_point = point

    return latitudes, longitudes, elevations, distances


def process_route(gpx_file_path):
    # Extract route data
    latitudes, longitudes, elevations, distances = extract_route_data(
        gpx_file_path)

    # Create a DataFrame
    data = {
        'Latitude': latitudes,
        'Longitude': longitudes,
        'Distance (m)': distances,
        'Elevation (m)': elevations
    }

    df_route = pd.DataFrame(data)

    # Calculate segment length and slope for each segment
    segment_lengths = [0]  # Initial segment length is 0 for the starting point
    slopes = [0]           # Initial slope is 0 for the starting point

    for i in range(1, len(df_route)):
        segment_length = df_route['Distance (m)'].iloc[i] - \
            df_route['Distance (m)'].iloc[i - 1]
        elevation_change = df_route['Elevation (m)'].iloc[i] - \
            df_route['Elevation (m)'].iloc[i - 1]
        slope = np.degrees(np.arctan(elevation_change /
                           segment_length)) if segment_length != 0 else 0

        segment_lengths.append(segment_length)
        slopes.append(slope)

    df_route['Segment Length (m)'] = segment_lengths
    df_route['Slope (Degrees)'] = slopes

    # Calculate fuel consumption for each segment
    df_route['Fuel Consumption (liters/100km)'] = df_route['Slope (Degrees)'].apply(
        get_fuel_consumption)

    # Calculate fuel consumption per segment in liters
    df_route['Fuel Consumption per Segment (liters)'] = (
        df_route['Fuel Consumption (liters/100km)'] * df_route['Segment Length (m)']) / 100000

    return df_route

# Function to create a folium map with routes


def create_route_map(df_route, df_alt_route):
    # Combine the latitudes and longitudes of both routes
    all_latitudes = df_route['Latitude'].tolist(
    ) + df_alt_route['Latitude'].tolist()
    all_longitudes = df_route['Longitude'].tolist(
    ) + df_alt_route['Longitude'].tolist()

    # Calculate the center of the bounding box
    center_lat = (max(all_latitudes) + min(all_latitudes)) / 2
    center_lon = (max(all_longitudes) + min(all_longitudes)) / 2

    # Create a folium map centered around the midpoint of the bounding box
    route_map = folium.Map(
        location=[center_lat, center_lon], zoom_start=14, tiles="Cartodb Positron")

    # Add alternative route to the map
    alt_route_coords = list(
        zip(df_alt_route['Latitude'], df_alt_route['Longitude']))
    folium.PolyLine(alt_route_coords, color='black', weight=5,
                    opacity=1, tooltip='Alternative Route').add_to(route_map)

    # Add primary route to the map
    primary_route_coords = list(
        zip(df_route['Latitude'], df_route['Longitude']))
    # folium.PolyLine(primary_route_coords, color='#0066FF', weight=2.5,
    #                 opacity=1, tooltip='Primary Route').add_to(route_map)


    folium.Marker(alt_route_coords[0], popup='Start: Alternative Route', icon=folium.Icon(
        color='black')).add_to(route_map)
    folium.Marker(alt_route_coords[-1], popup='End: Alternative Route',
                  icon=folium.Icon(color='black')).add_to(route_map)

    return route_map

In [4]:
# Example usage for primary and alternative routes
route_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\04_SC_NR\M3430\Route.gpx"
alt_route_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\04_SC_NR\M3430\Route_alt.gpx"

route_path = r"C:\Users\relia\Documents\GitHub\Bridges\04_SC_NR\M3430\Route.gpx"
alt_route_path = r"C:\Users\relia\Documents\GitHub\Bridges\04_SC_NR\M3430\Route_alt.gpx"

df_route = process_route(route_path)
df_alt_route = process_route(alt_route_path)

In [8]:
df_route["Fuel Consumption per Segment (liters)"].sum(), \
df_alt_route["Fuel Consumption per Segment (liters)"].sum()

(0.8058522578887617, 1.2309249882126614)

In [43]:
# Create the folium map with both routes
route_map = create_route_map(df_route, df_alt_route)

# Save the map to an HTML file
route_map.save('M3430_route_map.html')

# Display the map in the Jupyter notebook (if running in Jupyter)
route_map

In [37]:
clean_map = folium.Map(
    location=[48.7363, 19.1358], zoom_start=14, tiles="Cartodb Positron")
clean_map.save('clean_map.html')